# A Shell for clingo, using Multi Shot Solving

The task of this notebook is to implement a shell for clingo, based on the ideas of [1], using the clingo API described in [2]. 

Before starting the project, please read sections 1, 3 and 4 of [1] and sections 4 and 5 of [2].

You can use the script `clingo-shell.py` as a starting point for implementation.

[1] [Gebser, M., Obermeier, P., & Schaub, T. (2015). Interactive Answer Set Programming - Preliminary Report. CoRR, abs/1511.01261.](https://www.cs.uni-potsdam.de/wv/publications/DBLP_journals/corr/GebserOS15.pdf)

[2] [Kaminski, R., Romero, J., Schaub, T., & Wanko, P. (2023). How to Build Your Own ASP-based System?! Theory Pract. Log. Program., 23(1), 299–361.](https://arxiv.org/pdf/2008.06692.pdf)


## A quick note about externals and assumptions

In the following block, the external `a` is first false by default, then true, and finally open.

In [1]:
from clingo import Control
from clingo.symbol import parse_term

control = Control(["0"])
control.add("base", [], "#external a. b :- not a.") # a is external (false by default)
a = parse_term("a")
control.ground([("base", [])])
control.solve(on_model=print) # {b}

print()
control.assign_external(a, True) # a is true
control.solve(on_model=print) # {a}

print()
control.assign_external(a, None) # a is open
control.solve(on_model=print) # {b}, {a}

b

a

b
a


SolveResult(5)

We can simulate this behavior adding a choice rule for `{a}` and 
setting its value using assumptions:
* if the external `a` was false then we add the assumption that `a` is false
* if the external `a` was true then we add the assumption that `a` is true
* if the external `a` was open then we add no assumption

In [2]:
from clingo import Control
from clingo.symbol import parse_term

control = Control(["0"])
control.add("base", [], "{a}. b :- not a.") # choice rule for a
control.ground([("base", [])])
a = parse_term("a")
assumptions = [(a, False)] # assume a is false
control.solve(on_model=print, assumptions=assumptions) # {b}

print()
assumptions = [(a, True)] # assume a is true
control.solve(on_model=print, assumptions=assumptions) # {a}

print()
assumptions = [] # no assumption (a is open)
control.solve(on_model=print, assumptions=assumptions) # {b}, {a}

b

a

b
a


SolveResult(5)

With externals, we can also run 
```
control.release_external(parse_term("a"))
```
This makes `a` permanently false, and tells clingo to simplify the program taking into account this information.

We can simulate this adding `(a, False)` to our list of assumptions forever, but in this case no simplifications take place.

Externals can also be redefined. 

For example, if we run the next cell after the code with externals, we obtain the answer `{a}`.

But if we run the next cell after the code with assumptions, we obtain an error.

In [3]:
control.add("new", [], "a.")
control.ground([("new", [])])
control.solve(on_model=print) # {a} 
print()

b
a



We can also enable this kind of redefinition using assumptions, but the solution is a bit involved and we do not show it here.

We can also go in the other direction, and simulate assumptions using externals and additional programs.

For example, to simulate the assumption that `a` is false, we can:
* add the rule `:- a, flag1. #external flag1. [true]` enforcing that `a` is false
  given that `flag1` is made true
* solve
* release `flag1` to come back to the state before adding the rule

In [4]:
from clingo import Control
from clingo.symbol import parse_term

control = Control(["0"])
control.add("base", [], "{a}. b :- not a.")
control.ground([("base", [])])

control.add("assumption1", [], ":- a, flag1. #external flag1. [true]") # a is false
control.ground([("assumption1", [])])
control.solve(on_model=print) # {b}
control.release_external(parse_term("flag1"))

print()
control.add("assumption2", [], ":- not a, flag2. #external flag2. [true]") # a is true
control.ground([("assumption2", [])])
control.solve(on_model=print) # {a}
control.release_external(parse_term("flag2"))

print()
control.solve(on_model=print) # {b}, {a}

b flag1

a flag2

b
a


SolveResult(5)

## Task

The main difference between our approach and the approach presented in [1] is that we do not implement directly the command `query`. Instead, we use a combination of other commands to achieve a similar functionality.

In addition, we define two new groups of commands:
* `opt-on` and `opt-off` to turn on and off optimization (we assume it is off by default), and
* `show-add`, `show-remove`, `hide-add` and `hide-remove` to mantain a list of shown and hidden predicates that are used to determine what atoms are shown

The script should optionally accept a file including a sequence of commands, that should be run before starting the interaction.

In our simplest example, given the file `example/example.lp`:

In [5]:
! cat example/example.lp

dom(1..2).
1 { a(X) : dom(X) }.
b(X) :- a(X).
#show b/1.


You can run this sequence of commands:

In [6]:
! cat example/example-batch.txt

load example/example.lp
solve
exit


and obtain:

In [7]:
! python clingo-shell.py example/example-batch.txt

?- load example/example.lp

?- solve
Model: [b(2)]
SAT

?- exit


Your script should be able to simulate the approach of [1]. For this, the file `ncoloring.lp` of [1] is available at `ncoloring/ncoloring.lp`:

In [8]:
! cat ncoloring/ncoloring.lp 

#const n = 3.
#const m = 10.
color(1..n).

% Extract nodes from edges
node(X) :- edge(X,_).
node(X) :- edge(_,X).

% Generate n-coloring
1 { mark(X,C) : color(C) } 1 :- node(X).
:- edge(X,Y), mark(X,C), mark(Y,C).

% Allow aspic user to add and remove edges via externals
#external edge(X,Y) : X = 1..m-1, Y = X+1..m.

% Display n-coloring
#show mark/2.


In the directory `ncoloring` there is also a batch file simulating the commands used in the examples of [1], and adding a few more at the end:

In [9]:
! cat ncoloring/ncoloring-batch.txt

load ncoloring.lp
solve
assert edge(1,2)
assert edge(1,4)
assert edge(2,3)
assert edge(3,4)
assume mark(1,1)
solve
cancel mark(1,1)
option -n 0
assume mark(1,1)
solve
cancel mark(1,1)
option -e brave
assume mark(1,1)
solve
cancel mark(1,1)
option -e cautious
assume mark(1,1)
solve
cancel mark(1,1)
option -e auto
add query1 :- mark(1,1), 1 { mark(3,2); not mark(4,2) }.
assume query1
solve
cancel query1
assert edge(2,4)
assume query1
solve
cancel query1
open edge(2,4)
assume query1
solve
cancel query1
retract edge(2,4)
assume query1
solve
cancel query1
assume not mark(2,3)
assume mark(1,1)
solve
cancel mark(1,1)
assume query1
solve
cancel query1
cancel not mark(2,3)
add #external elim(X,C) : X = 1..m, color(C).
add :- edge(X,Y), elim(X,C), mark(Y,C).
add :- edge(X,Y), mark(X,C), elim(Y,C).
assert elim(2,3)
assert elim(4,2)
add query2 :- mark(X,1), elim(X,C).
assume query2
solve
cancel query2
add query3 :- mark(X,C), elim(X,C).
assume query3
solve
cancel query3
add #minimize{ C@X : mark(X

Observe how the `query` commands are simulated by a mixture of:
* `assume` and `cancel` commands, and
* `add` commands to add rules defining new atoms `query1`, `query2`, ..., when needed.

The output generated by your script should look like this:

In [10]:
! cat ncoloring/ncoloring.out

?- load ncoloring.lp

?- solve
Model: []
SAT

?- assert edge(1,2)

?- assert edge(1,4)

?- assert edge(2,3)

?- assert edge(3,4)

?- assume mark(1,1)

?- solve
Model: [mark(1,1), mark(2,3), mark(3,2), mark(4,3)]
SAT

?- cancel mark(1,1)

?- option -n 0

?- assume mark(1,1)

?- solve
Model: [mark(1,1), mark(2,3), mark(3,2), mark(4,3)]
Model: [mark(1,1), mark(2,3), mark(3,1), mark(4,3)]
Model: [mark(1,1), mark(2,2), mark(3,1), mark(4,3)]
Model: [mark(1,1), mark(2,3), mark(3,1), mark(4,2)]
Model: [mark(1,1), mark(2,2), mark(3,1), mark(4,2)]
Model: [mark(1,1), mark(2,2), mark(3,3), mark(4,2)]
SAT

?- cancel mark(1,1)

?- option -e brave

?- assume mark(1,1)

?- solve
Model: [mark(1,1), mark(2,2), mark(2,3), mark(3,1), mark(3,2), mark(3,3), mark(4,2), mark(4,3)]
SAT

?- cancel mark(1,1)

?- option -e cautious

?- assume mark(1,1)

?- solve
Model: [mark(1,1)]
SAT

?- cancel mark(1,1)

?- option -e auto

?- add query1 :- mark(1,1), 1 { mark(3,2); not mark(4,2) }.

?- assume query1

?- solve
M

Observe that after adding the minimize statement, the shell still returns all 4 (possibly non-optimal) answer sets.

When `opt-on` is issued, it returns the two optimal answer sets.

Then, `mark(2,2)` is assumed to be false, and the shell returns other two optimal answer sets.

When `opt-off` is executed, the shell returns again the 4 (possibly non-optimal) answer sets.

## Hints

* You can start from the script `clingo-shell.py`.
* To work step-by-step with the file `ncoloring/ncoloring-batch.txt`,
  you may want to add a `break` command after the `print("Unknown command")` at the end of the `run()` function.
* You can check your script doing a `diff` between the output of `python clingo-shell.py ncoloring/ncoloring-batch.txt` and `ncoloring/ncoloring.out`.
* You can parse the commands with Python's `split()` function.
  Do not worry about printing nice error messages or warnings.
* These are the attributes and functions that we used to implement the shell:
  * clingo.symbol.parse_term()
  * clingo.symbol.Symbol: 
    - name, arguments 
  * clingo.control.Control:
    - configuration.solve.models, configuration.solve.enum_mode, configuration.solve.opt_mode
    - load(), add(), ground(), solve(), release_external(), assign_external()
  * clingo.solving.Model:
    - optimality_proven
    - symbols()
* Feel free to modify/extend the shell as you wish!